랜덤 포레스트 알고리즘에 rfe(recurrent feature elimination)를 정의한 코드

In [1]:
import pandas as pd

In [3]:
def rf_rfe(x_data, X_train, y_train, X_test, y_test, method, ratio=0.09, min_feats=40) :
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns = ['model', 'n_feats', 'feats', 'rmse', 'accuracy'])
    while True :
        model = RandomForestClassifier()
        model.fit(X_train[feats], y_train)
        pred = model.predict(X_test[feats])
        rmse = rmse(pred, y_test)
        accuracy = accuracy_score(y_test, pred)
        n_feats = len(feats)
        print(n_feats, rmse, accuracy)
        archive = archive.append({'model':model, 'n_feats':n_feats, 'feats':feats, 'rmse':rmse, 'accuracy':accuracy}, ignore_index=True)
        if method == 'perm' :
            perm = PermutationImportance(model, random_state=0).fit(X_test[feats], y_test)
            feat_imp = pd.Series(perm.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats :
            break
        else : 
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    
    archive_rf = pd.DataFrame(archive)
    archive_rf = archive_rf.sort_values(by = "rmse")
    filename = f"randomforest_{method}.csv"
    archive_rf.to_csv(filename, encoding = 'utf-8-sig')
    
    return archive